In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the data
df = pd.read_excel('tuber.xlsx', sheet_name='исходные данные')

In [ ]:
df['вес'].dropna()

# Удаление пропусков
1. выбираем исследуемый признак для пропусков
2. получаем индексы
3. удаляем из датафрейма

In [ ]:
missing_indices = df[df['к/дней'].isna()].index # получаем индексы пропусков
print(f"Индексы пропущенных значений: {missing_indices}")

df_dropped = df.dropna(subset=['к/дней']).copy()

df_dropped

In [ ]:
df_dropped = df.drop(missing_indices).copy()

df_dropped

# Заполнение медианой и средним
1. выбираем исследуемый признак
2. считаем статистику
3. заполняем
4. проверяем, что переменная не искажает результаты

In [ ]:
df_filled_mean = df.copy()
df_filled_median = df.copy()

# Заполнение пропусков средним значением
mean_value = df['возраст'].mean()
df_filled_mean['возраст'].fillna(mean_value, inplace=True)

# Заполнение пропусков медианой
median_value = df['возраст'].median()
df_filled_median['возраст'].fillna(median_value, inplace=True)



In [ ]:
# Получение описательных статистик для каждого DataFrame
desc_original = df['возраст'].describe()
desc_filled_mean = df_filled_mean['возраст'].describe()
desc_filled_median = df_filled_median['возраст'].describe()

# Объединение описательных статистик в одну таблицу
desc_table = pd.concat([desc_original, desc_filled_mean, desc_filled_median], axis=1)
desc_table.columns = ['Original', 'Filled with Mean', 'Filled with Median']
desc_table

# Интерполяция и экстраполяция
1. Работет с временными рядами
2. Интерполяция - поиск на основе прошлого и следующего значения
3. Экстраполяция - поиск следующих на основе уже известных

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Создаем искусственные данные
np.random.seed(42)
months = np.array(['Январь', 'Февраль', 'Март', 'Апрель', 'Май', 'Июнь', 
                   'Июль', 'Август', 'Сентябрь', 'Октябрь', 'Ноябрь', 'Декабрь'])
sales = 100 + 20 * np.sin(np.linspace(0, 2 * np.pi, 12)) + 10 * np.random.normal(size=12)

# Искусственно создаем пропущенные значения
sales[3] = np.nan
sales[7] = np.nan

df_inter = pd.DataFrame({'Месяц': months, 'Продажи': sales})

# Заполняем пропущенные значения через интерполяцию
df_inter['Интерполированные продажи'] = df_inter['Продажи'].interpolate(method='linear')

# График
plt.figure(figsize=(12,6))
plt.bar(df_inter['Месяц'], df_inter['Продажи'], label='Оригинальные продажи с пропущенными значениями')
plt.bar(df_inter['Месяц'], df_inter['Интерполированные продажи'], label='Интерполированные продажи', color='orange', alpha=0.5)
plt.xticks(rotation=45)
plt.legend()
plt.show()

In [ ]:
df_inter[['Месяц', 'Продажи']]

In [ ]:
from sklearn.linear_model import LinearRegression

# Подготовка данных для экстраполяции
X = np.array(range(0, 12)).reshape(-1, 1) # месяцы
y = df_inter['Интерполированные продажи'].values
# Создание модели линейной регрессии
model = LinearRegression()
model.fit(X, y)

# Экстраполяция на следующий год
X_future = np.array(range(12, 24)).reshape(-1, 1)
sales_future = model.predict(X_future)

# Добавляем экстраполированные данные в наш датасет
df_future = pd.DataFrame({'Месяц': months, 'Продажи': sales_future})

# График
plt.figure(figsize=(12,6))
plt.bar(df_future['Месяц'], df_future['Продажи'], label='Продажи')
plt.xticks(rotation=45)
plt.legend()
plt.show()

# Заполнение пропусков на основе корреляции

In [ ]:
import pandas as pd
import numpy as np

def fill_missing_values(df, target_column, other_columns):
    df = df.copy()
    # Фильтрация данных, чтобы убрать строки, где целевая колонка уже заполнена
    df_missing = df[df[target_column].isna()]
    
    # Среднее значение целевой колонки
    target_mean = df[target_column].mean()
    
    for idx, row in df_missing.iterrows():
        weighted_sums = 0
        correlation_sums = 0
        for col in other_columns:
            # Пропускаем, если в текущей строке пропущенное значение в одной из other_columns
            if np.isnan(row[col]):
                continue
            
            # Среднее значение для текущей колонки из other_columns
            col_mean = df[col].mean()
            
            # Коэффициент корреляции между target_column и текущей колонкой
            correlation = df[target_column].corr(df[col])
            
            # Вычисляем взвешенное значение
            weighted_value = correlation * (row[col] - col_mean)
            
            # Обновляем суммы
            weighted_sums += weighted_value
            correlation_sums += abs(correlation)
        
        # Вычисляем прогнозируемое значение для пропущенного значения
        if correlation_sums != 0:
            predicted_value = target_mean + weighted_sums / correlation_sums
        else:
            predicted_value = target_mean
        
        # Заполняем пропущенное значение
        df.at[idx, target_column] = predicted_value
    
    return df



In [ ]:
df[['к/дней','рост', 'вес', 'возраст']].corr()

In [ ]:
df_filled_by_corr = fill_missing_values(_, 'к/дней', ['рост', 'вес', 'возраст'])

In [ ]:
df['к/дней'].describe()

In [ ]:
df_filled_by_corr['к/дней'].describe()

In [ ]:
def plot_distributions(before, after, column_name):
    plt.figure(figsize=(14, 6))

    plt.subplot(1, 2, 1)
    sns.histplot(before[column_name], kde=False, bins=10)
    plt.title('Распределение до заполнения')

    plt.subplot(1, 2, 2)
    sns.histplot(after[column_name], kde=False, bins=10)
    plt.title('Распределение после заполнения')

    plt.show()

plot_distributions(df, df_filled_by_corr, 'к/дней')

# KNN на основе ближайших соседей (расстояние)

In [ ]:
df[['возраст','к/дней', 'рост', 'вес']]

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
def euclidean_distance(x, y):
    return np.sqrt(np.sum((x - y)**2))

def knn_impute(df, target_col, k=3):
    df = df.copy()
    missing_indices = df.index[df[target_col].isnull()].tolist()
    
    for i in missing_indices:
        row = df.loc[i]
        non_missing_df = df.dropna()
        distances = []
        
        for j, other_row in non_missing_df.iterrows():
            dist = euclidean_distance(row.dropna(), other_row.loc[row.dropna().index])
            distances.append((j, dist))

        neighbors = sorted(distances, key=lambda x: x[1])[:k]
        neighbor_indices = [index for index, _ in neighbors]
        neighbor_df = df.iloc[neighbor_indices]
        
        if pd.isna(row[target_col]):
            mean_val = neighbor_df[target_col].mean()
            df.at[i, target_col] = mean_val

    return df

df_distance = df[['возраст','к/дней', 'рост', 'вес']]

scaler = StandardScaler()
scaler.fit(df_distance)
df_scaled = pd.DataFrame(scaler.transform(df_distance), columns=df_distance.columns, index=df_distance.index)

In [ ]:
df_distance

In [ ]:
# Сохранение индексов пропущенных значений
missing_indices = df_distance.index[df_distance.isnull().any(axis=1)].tolist()

# Визуализация до KNN импутации
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.scatter(df_distance['возраст'], df_distance['рост'], c='b')
plt.scatter(df_distance['возраст'][df_distance['возраст'].isna()], df_distance['рост'][df_distance['возраст'].isna()], c='r', s=100)
plt.title("Before KNN Imputation")
plt.xlabel("возраст")
plt.ylabel("рост")

# Применение KNN импутации
df_impute = knn_impute(df_scaled, 'возраст')
df_impute = pd.DataFrame(scaler.inverse_transform(df_impute), columns=df_distance.columns, index=df_distance.index)

# Визуализация после KNN импутации
plt.subplot(1, 2, 2)
plt.scatter(df_impute['возраст'], df_impute['рост'], c='g')
plt.scatter(df_impute.loc[missing_indices, 'возраст'], df_impute.loc[missing_indices, 'рост'], c='r', s=100)
plt.title("After KNN Imputation")
plt.xlabel("возраст")
plt.ylabel("рост")

plt.show()

# Поиск пропусков в категориях

## Мода и категориальные признаки

In [ ]:
df = pd.read_excel('tuber.xlsx', sheet_name='исходные данные')
df

In [ ]:
df.info()

In [ ]:
print(df.пол.unique())
df.groupby('Режимы химиотерапии')['пол']\
    .apply(lambda x: x.mode().iloc[0])

## KNN

In [ ]:
from sklearn.impute import KNNImputer
import numpy as np
import pandas as pd

df_cat = pd.DataFrame({
    'Age': [25, 30, 35, 40, 45],
    'Gender': ['Male', 'Female', np.nan, 'Male', np.nan],
    'Salary': [50000, 55000, 60000, 65000, 70000]
})

# LabelEncoding
df_cat['Gender'] = df_cat['Gender'].map({'Male': 0, 'Female': 1})

knn_imputer = KNNImputer(n_neighbors=3)
df_imputed = knn_imputer.fit_transform(df_cat)

df_imputed = pd.DataFrame(df_imputed, columns=df_cat.columns)

df_imputed

# Методы трансформации

## интерактивные признаки

In [ ]:
df['BMI'] = df['вес'] / df['рост'].apply(lambda x: x**2)
df['BMI']

## Масштабирование

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

minmax_scaler = MinMaxScaler()
standard_scaler = StandardScaler()

# Apply MinMax scaling
df['возраст_minmax'] = minmax_scaler.fit_transform(df[['возраст']])

# Apply Standard scaling
df['возраст_standard'] = standard_scaler.fit_transform(df[['возраст']])

df[['возраст', 'возраст_minmax', 'возраст_standard']]

## Бининг

In [ ]:
df_income = pd.DataFrame({
    'ID': [1, 2, 3, 4, 5],
    'Income': [20000, 40000, 60000, 120000, 180000]
})

# Define income bins and labels
bins = [20000, 50000, 100000, 200000]
labels = ['Low', 'Medium', 'High']
df_income['Income_Category'] = pd.cut(df_income['Income'], bins=bins, labels=labels, right=False)

df_income

In [ ]:
from sklearn.linear_model import LinearRegression
import numpy as np

# Создаем искусственные данные
X = np.array([
    [25, 50000],
    [30, 55000],
    [35, 60000],
    [40, 100000],
    [45, 110000],
    [50, 105000]
])
y = np.array([0, 1, 0, 1, 1, 0])

# Обучаем модель без масштабирования
model = LinearRegression()
model.fit(X, y)

print("Веса без масштабирования: ", model.coef_)

In [ ]:
from sklearn.preprocessing import StandardScaler

# Масштабируем признаки
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Обучаем модель с масштабированными данными
model_scaled = LinearRegression()
model_scaled.fit(X_scaled, y)

print("Веса с масштабированием: ", model_scaled.coef_)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

df = pd.read_excel('tuber.xlsx', sheet_name='исходные данные')
df = df[['к/дней', 'пол', 'возраст', 'Алкоголь']].dropna()

# Замените значения в столбце 'пол' на 1, если это 'ж', иначе 0
df['пол'] = df['пол'].apply(lambda x: 1 if x == 'ж' else 0)
df['возраст'] = df['возраст'].astype(float)
# Замените значения в столбце 'Алкоголь' на 1, если содержит 'да', иначе 0
df['Алкоголь'] = df['Алкоголь'].apply(lambda x: 1 if 'да' in x else 0)
scaler = MinMaxScaler()
df['возраст'] = scaler.fit_transform(df[['возраст']])
df

In [ ]:
# Разделяем признаки и целевую переменную
X = df[['возраст', 'пол']]
y = df['к/дней']
y = y.values.reshape(-1, 1)

# Разделяем данные на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Линейная регрессия
model = LinearRegression()
model.fit(X_train, y_train)

# Выводим коэффициенты
print('Коэффициенты:')
print(model.coef_)

# Предсказываем результат на тестовом наборе
y_pred = model.predict(X_test)

# Оцениваем модель по MAE
mae = mean_absolute_error(y_test, y_pred)
print(f'Средняя абсолютная ошибка (MAE): {mae:.2f}')

In [ ]:
# Разделяем признаки и целевую переменную
X = pd.get_dummies(X, columns=['пол'])
y = df['к/дней']
y = y.values.reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Линейная регрессия
model = LinearRegression()
model.fit(X_train, y_train)

# Выводим коэффициенты
print('Коэффициенты:')
print(model.coef_)

# Предсказываем результат на тестовом наборе
y_pred = model.predict(X_test)

# Оцениваем модель по MAE
mae = mean_absolute_error(y_test, y_pred)
print(f'Средняя абсолютная ошибка (MAE): {mae:.2f}')

In [ ]:
df_alco = df['Алкоголь'].copy().dropna()
alco = pd.get_dummies(df_alco).astype(int)

In [ ]:
alco

In [ ]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(alco)

In [ ]:
enc.categories_

In [2]:
import pandas as pd
df = pd.read_excel('tuber.xlsx', sheet_name='исходные данные')


0        м
1        ж
2        ж
3        м
4        м
      ... 
398    NaN
399    NaN
400    NaN
401    NaN
402    NaN
Name: пол, Length: 403, dtype: object

In [6]:
df['пол'].dropna()

0      м
1      ж
2      ж
3      м
4      м
      ..
356    ж
357    м
358    ж
359    ж
360    м
Name: пол, Length: 361, dtype: object

In [8]:
pd.get_dummies(df['пол'].dropna()).astype(int)

,ж,м,мм
0,0,1,0
1,1,0,0
2,1,0,0
3,0,1,0
4,0,1,0
...,...,...,...
356,1,0,0
357,0,1,0
358,1,0,0
359,1,0,0
